# Speed comparison

In this notebook, we will apply Neural Fine Gray and Desurv on the FRAMINGHAM data and measure the execution times

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../DeepSurvivalMachines/')

### Load the FRAMINGHAM Dataset

The package includes helper functions to load the dataset.

X represents an np.array of features (covariates),
T is the event/censoring times and,
E is the censoring indicator.

In [ ]:
from nfg import datasets
x, t, e, columns = datasets.load_dataset('FRAMINGHAM', competing = True)

### Compute horizons at which we evaluate the performance of Neural Fine Gray

Survival predictions are issued at certain time horizons. Here we will evaluate the performance
of NFG to issue predictions at the 25th, 50th and 75th event time quantile as is standard practice in Survival Analysis.

In [ ]:
import numpy as np
import torch
np.random.seed(42)
torch.random.manual_seed(42)

horizons = [0.25, 0.5, 0.75]
times = np.quantile(t[e > 0], horizons) # Fixed horizons for accurate comparison between competing and non competing

In [ ]:
# Display the percentage of observed event at different time horizon
for time in times:
    print('At time {:.2f}'.format(time))
    for risk in np.unique(e):
        print('\t {:.2f} % observed risk {}'.format(100 * ((e == risk) & (t < time)).mean(), risk))

In [ ]:
for risk in np.unique(e):
    print('\t {:.2f} % observed risk {}'.format(100 * ((e[e!=0] == risk)).mean(), risk))

### Splitting the data into train, test and validation sets

We will train NSC on 80% of the Data (10 % of which is used for stopping criterion and 10% for model Selection) and report performance on the remaining 20% held out test set.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

x_train, x_test, t_train, t_test, e_train, e_test = train_test_split(x, t, e, test_size = 0.2, random_state = 42)
x_train, x_val, t_train, t_val, e_train, e_val = train_test_split(x_train, t_train, e_train, test_size = 0.2, random_state = 42)
x_dev, x_val, t_dev, t_val, e_dev, e_val = train_test_split(x_val, t_val, e_val, test_size = 0.5, random_state = 42)

ss = MinMaxScaler().fit(t_train.reshape(-1, 1))
t_train_ddh = ss.transform(t_train.reshape(-1, 1)).flatten()
t_dev_ddh = ss.transform(t_dev.reshape(-1, 1)).flatten()
t_val_ddh = ss.transform(t_val.reshape(-1, 1)).flatten()
times_ddh = ss.transform(np.array(times).reshape(-1, 1)).flatten()

### Setting the parameter grid

Lets set up the parameter grid to tune hyper-parameters. We will tune the number of underlying survival distributions, 
($K$), the distribution choices (Log-Normal or Weibull), the learning rate for the Adam optimizer between $1\times10^{-3}$ and $1\times10^{-4}$ and the number of hidden layers between $0, 1$ and $2$.

In [ ]:
from sklearn.model_selection import ParameterSampler

In [ ]:
layers = [[50, 50, 50]]
param = {
            'learning_rate' : [1e-3],
            'layers_surv': layers,
            'layers' : layers,
            'batch': [100],
            }
n_iter = 500

### Model Training and Selection

In [ ]:
import time

In [ ]:
from nfg import NeuralFineGray
from desurv import DeSurv

In [ ]:
models = []
desurv_times, fine_times = [], []
# Ensures that we use the same architecture
for _ in range(100):
    desurv_time = time.process_time()
    model = DeSurv(layers = param['layers'], layers_surv = param['layers_surv'], dropout = 0.25) 
    model.fit(x_train, t_train_ddh, e_train, n_iter = n_iter, bs = param['batch'], patience_max = n_iter, # Ensures that we train for n_iter iterations
            lr = param['learning_rate'], val_data = (x_dev, t_dev_ddh, e_dev))
    nll = model.compute_nll(x_val, t_val_ddh, e_val)
    desurv_times.append(((time.process_time() - desurv_time) / n_iter, nll))

    fine_time = time.process_time()
    model = NeuralFineGray(layers = param['layers'], layers_surv = param['layers_surv'], dropout = 0.25)
    model.fit(x_train, t_train_ddh, e_train, n_iter = n_iter, bs = param['batch'], patience_max = n_iter,
            lr = param['learning_rate'], val_data = (x_dev, t_dev_ddh, e_dev))
    nll = model.compute_nll(x_val, t_val_ddh, e_val)
    fine_times.append(((time.process_time() - fine_time) / n_iter, nll))

# Display performance and time 

In [ ]:
import pandas as pd
import seaborn as sns
sns.set_theme()

In [ ]:
desurv_times = pd.DataFrame(desurv_times, columns = ['Second / Epoch', 'NLL'])
fine_times = pd.DataFrame(fine_times, columns = ['Second / Epoch', 'NLL'])

In [ ]:
ax = desurv_times.plot.scatter('Second / Epoch', 'NLL', c = 'blue', alpha = 0.5, label = 'DeSurv')
desurv_times.mean().to_frame().T.plot.scatter('Second / Epoch', 'NLL', c = 'blue', s = 50, marker = 'x', ax = ax)

fine_times.plot.scatter('Second / Epoch', 'NLL', ax = ax, color = 'red', alpha = 0.5, label = 'Fine Gray')
fine_times.mean().to_frame().T.plot.scatter('Second / Epoch', 'NLL', c = 'red', s = 50, marker = 'x', ax = ax)

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))